In [ ]:
!pip install -U awscli boto3 sagemaker rich watermark xgboost scikit-learn --quiet

In [1]:
%load_ext watermark
%load_ext rich

%watermark -p awscli,boto3,sagemaker,xgboost,sklearn

awscli   : 1.27.99
boto3    : 1.26.99
sagemaker: 2.141.0
xgboost  : 1.7.4
sklearn  : 1.2.2



### Upload model artifacts as `model.tar.gz` to S3


In [2]:
import tarfile
import os

model_path = os.path.join("./models", "xgboost-model")
model_output_path = os.path.join("./models", "model.tar.gz")

if not os.path.exists(model_output_path):
    print(f"Compressing model to {model_output_path}")
    tar = tarfile.open(model_output_path, "w:gz")
    tar.add(model_path, arcname="xgboost-model")
    tar.close()
else:
    print(f"Model file exists: {model_output_path}")

Model file exists: ./models/model.tar.gz


In [3]:
import sagemaker
import boto3
from sagemaker.s3 import S3Uploader, s3_path_join
from sagemaker import session
from sagemaker import get_execution_role
from rich import print

sm_session = session.Session()
region = sm_session._region_name
role = get_execution_role()
bucket = sm_session.default_bucket()
sm_client = boto3.client("sagemaker")

prefix = "sagemaker/abalone"
account_id = boto3.client("sts").get_caller_identity().get("Account")
model_s3uri = s3_path_join(f"s3://{bucket}/{prefix}", "models/byoc-serial-inference/xgbpredictor")

print(f"Role: {role}")
print(f"Bucket: {bucket}")
print(f"Model base: {model_s3uri}")

S3Uploader.upload(
    local_path=model_output_path,
    desired_s3_uri=model_s3uri,
    sagemaker_session=sm_session,
)

Role: arn:aws:iam::726793866085:role/service-role/AmazonSageMaker-ExecutionRole-20220313T104021

Bucket: sagemaker-us-east-1-726793866085

Model base: s3://sagemaker-us-east-1-726793866085/sagemaker/abalone/models/byoc-serial-inference/xgbpredictor

's3://sagemaker-us-east-1-726793866085/sagemaker/abalone/models/byoc-serial-inference/xgbpredictor/model.tar.gz'

### Create model object with custom inference image

In [4]:
from datetime import datetime
from uuid import uuid4

image_name = "byoc/abalone-predictor"
ecr_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"
suffix = f"{str(uuid4())[:5]}-{datetime.now().strftime('%d%b%Y')}"

model_data_url = s3_path_join(model_s3uri, "model.tar.gz")

print(f"model_image_uri: {ecr_image}")
model_name = f"AbaloneXGB-predictor-{suffix}"

print(f"Creating model : {model_name} with {model_data_url}")
model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": ecr_image, "ModelDataUrl": model_data_url},
)

print(f"Response")
print(model_response["ModelArn"])

model_image_uri: 726793866085.dkr.ecr.us-east-1.amazonaws.com/byoc/abalone-predictor:latest

Creating model : AbaloneXGB-predictor-12926-26Mar2023 with 
s3://sagemaker-us-east-1-726793866085/sagemaker/abalone/models/byoc-serial-inference/xgbpredictor/model.tar.gz

Response

arn:aws:sagemaker:us-east-1:726793866085:model/abalonexgb-predictor-12926-26mar2023

### Create Endpoint Config

In [5]:
endpoint_config_name = f"AbaloneXGB-epc-predictor-{suffix}"
print(f"Creating endpoint config : {endpoint_config_name}")

# create endpoint config with data capture config using sagemaker boto3 client
epc_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.2xlarge",
            "InitialVariantWeight": 1,
        }
    ],
)

print(epc_response["EndpointConfigArn"])

Creating endpoint config : AbaloneXGB-epc-predictor-12926-26Mar2023

arn:aws:sagemaker:us-east-1:726793866085:endpoint-config/abalonexgb-epc-predictor-12926-26mar2023

In [6]:
# create endpoint with endpoint config and endpoint_image_uri
endpoint_name = f"AbaloneXGB-ep-predictor-{suffix}"
print(f"Creating endpoint : {endpoint_name}")

ep_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print(ep_response["EndpointArn"])

Creating endpoint : AbaloneXGB-ep-predictor-12926-26Mar2023

arn:aws:sagemaker:us-east-1:726793866085:endpoint/abalonexgb-ep-predictor-12926-26mar2023

In [7]:
from rich import print

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print(f"Endpoint [b]{endpoint_name}[/b] Status: [i]{status}[/i]")
# Get the waiter object
waiter = sm_client.get_waiter("endpoint_in_service")
# Apply the waiter on the endpoint
waiter.wait(EndpointName=endpoint_name)
# Get endpoint status using describe endpoint
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print(f"Endpoint [b]{endpoint_name}[/b] Status: [i]{status}[/i]")

Endpoint AbaloneXGB-ep-predictor-12926-26Mar2023 Status: Creating

Endpoint AbaloneXGB-ep-predictor-12926-26Mar2023 Status: InService

### Send test inference

In [ ]:
row = "0.5475261886982955,0.3215419286350158,0.24426074957028274,0.08695002906785124,0.11869309488761703,0.31322854300540076,0.041671324149052995,0.0,0.0,1.0"

In [11]:
import os
import json
import numpy as np
import pandas as pd
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from time import time, sleep

runtime_sm_client = boto3.client("sagemaker-runtime")

LOCALDIR = "."

local_test_dataset = f"{LOCALDIR}/test.csv"

test_s3uri = s3_path_join(f"s3://{bucket}/{prefix}", "data/test/test.csv")

# Download test data from S3
if not os.path.exists(local_test_dataset):
    print(f"Downloading test data to {LOCALDIR} from {test_s3uri} ...")
    S3Downloader.download(test_s3uri, local_path=LOCALDIR, sagemaker_session=sm_session)
else:
    print(f"Test file: {local_test_dataset} exists!")

limit = 10
i = 0

with open(local_test_dataset, "r") as _f:
    for row in _f:
        if i == 0:
            print(f"Headers")
            print(row)
            print("---" * 20)
            i += 1
        elif i <= limit:
            row = row.rstrip("\n")
            splits = row.split(",")
            # Remove the target column (first column)
            label = splits.pop(0)
            input_cols = ",".join(s for s in splits)
            prediction = None
            try:
                print(f"Invoking EP with record")
                # print(input_cols)
                prediction = runtime_sm_client.invoke_endpoint(
                    EndpointName=endpoint_name,
                    ContentType="text/csv",
                    Body=input_cols,
                )
                # print(prediction["Body"].read())
                response = prediction["Body"].read().decode("utf-8")
                # print(response)
                print(f"True: {label} | Predicted: {response}")
                i += 1
                sleep(0.5)
            except Exception as e:
                print(f"Prediction error: {e}")
                pass

Test file: ./test.csv exists!

Headers

n_Rings,n__Length,n__Diameter,n__Height,n__Whole weight,n__Shucked weight,n__Viscera weight,n__Shell 
weight,c__Sex_F,c__Sex_I,c__Sex_M

------------------------------------------------------------

Invoking EP with record

...

True: 9.0 | Predicted: {"result": [7.989439964294434]}

Invoking EP with record

True: 14.0 | Predicted: {"result": [9.012625694274902]}

Invoking EP with record

True: 11.0 | Predicted: {"result": [8.170607566833496]}

In [12]:
from datetime import timedelta

logs_client = boto3.client("logs")
end_time = datetime.utcnow()
start_time = end_time - timedelta(minutes=15)

log_group_name = f"/aws/sagemaker/Endpoints/{endpoint_name}"
log_streams = logs_client.describe_log_streams(logGroupName=log_group_name)
log_stream_name = log_streams["logStreams"][0]["logStreamName"]

# Retrieve the logs
logs = logs_client.get_log_events(
    logGroupName=log_group_name,
    logStreamName=log_stream_name,
    startTime=int(start_time.timestamp() * 1000),
    endTime=int(end_time.timestamp() * 1000),
)

# Print the logs
for event in logs["events"]:
    print(f"{datetime.fromtimestamp(event['timestamp'] // 1000)}: {event['message']}")

2023-03-26 20:04:56: [2023-03-26 20:04:55 +0000] [10] [INFO] Starting gunicorn 20.1.0

2023-03-26 20:04:56: [2023-03-26 20:04:55 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)

2023-03-26 20:04:56: [2023-03-26 20:04:55 +0000] [10] [INFO] Using worker: sync

2023-03-26 20:04:56: [2023-03-26 20:04:55 +0000] [12] [INFO] Booting worker with pid: 12

2023-03-26 20:04:56: [2023-03-26 20:04:56 +0000] [20] [INFO] Booting worker with pid: 20

...

2023-03-26 20:08:36: 169.254.178.2 - - [26/Mar/2023:20:08:35 +0000] "POST /invocations HTTP/1.1" 200 31 "-" 
"AHC/2.0"

2023-03-26 20:08:36: Transformed data: <xgboost.core.DMatrix object at 0x7f662be94970>

2023-03-26 20:08:36: Input: 
0.7579031464111207,0.676409429935641,0.8484424152684789,0.744922005460857,0.8995033322936672,0.7153794250135053,0.5
580240645341612,1.0,0.0,0.0

2023-03-26 20:08:36: <class 'numpy.ndarray'>

2023-03-26 20:08:36: <class 'xgboost.core.DMatrix'>

### Cleanup

In [13]:
# Delete endpoint, endpoint_configuration and model
print(f"EP: {endpoint_name}\nEPC: {endpoint_config_name}\nModel: {model_name}")

try:
    print(f"Deleting endpoint: {endpoint_name}")
    sm_client.delete_endpoint(EndpointName=endpoint_name)
except Exception as e:
    print(f"Error deleting EP: {endpoint_name}\n{e}")
    pass

try:
    print(f"Deleting endpoint_config: {endpoint_config_name}")
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
except Exception as e:
    print(f"Error deleting EPC: {endpoint_config_name}\n{e}")
    pass

try:
    print(f"Deleting Model: {model_name}")
    sm_client.delete_model(ModelName=model_name)
except Exception as e:
    print(f"Error deleting Model: {model_name}\n{e}")
    pass

EP: AbaloneXGB-ep-predictor-12926-26Mar2023
EPC: AbaloneXGB-epc-predictor-12926-26Mar2023
Model: AbaloneXGB-predictor-12926-26Mar2023

Deleting endpoint: AbaloneXGB-ep-predictor-12926-26Mar2023

Deleting endpoint_config: AbaloneXGB-epc-predictor-12926-26Mar2023

Deleting Model: AbaloneXGB-predictor-12926-26Mar2023